# 🔧 에이전트를 Function 도구로 사용하기

## 🎯 개요

이 실습에서는 **에이전트를 Function 도구로 사용**하는 방법을 알아봅니다.  
한 에이전트가 다른 에이전트를 도구로 호출하여 더욱 강력한 워크플로우를 구축할 수 있습니다.

---

## 🛠️ 에이전트를 Function 도구로 변환하기

### ✨ 핵심 메서드: `.as_tool()`

`ChatAgent`를 `.as_tool()` 메서드로 호출하면 다른 에이전트에 **도구로 제공**할 수 있습니다.

#### 📋 구현 단계
1. **Function 도구 생성**: 에이전트가 사용할 기본 함수 도구를 정의
2. **에이전트 생성**: Function 도구를 포함한 `ChatAgent` 생성
3. **도구로 변환**: `.as_tool()`을 사용하여 에이전트를 다른 에이전트의 도구로 제공

In [ ]:
from typing import Annotated
from pydantic import Field

## 날씨 Function 도구 정의
def get_weather(
    location: Annotated[str, Field(description="날씨 정보를 가져올 위치입니다. 📍")],
) -> str:
    """주어진 위치의 날씨를 가져옵니다. ☀️🌧️"""
    return f"{location}은 현재 구름이 많으며, 온도는 15°C 입니다."

## 날씨 에이전트 생성
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

weather_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="WeatherAgent",
    description="날씨에 대한 질문에 답변하는 에이전트입니다. ☁️🌡️",
    instructions="당신은 날씨에 대한 질문에 답변합니다. ☀️",
    tools=get_weather
)

## 🤖 메인 에이전트에 통합하기

메인 에이전트를 생성하고 **`weather_agent.as_tool()`** 을 호출하여 날씨 에이전트를 Function 도구로 제공합니다.

In [ ]:
main_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖💬",
    tools=weather_agent.as_tool()
)

result = await main_agent.run("서울의 날씨는 어떤가요? 🌆☀️")
print(result.text)

## ⚙️ 도구 커스터마이징

에이전트를 도구로 변환할 때 다음 속성들을 **사용자 정의**할 수 있습니다:

- **`name`**: 도구 이름
- **`description`**: 도구 설명
- **`arg_name`**: 인수 이름
- **`arg_description`**: 인수 설명

In [ ]:
# 에이전트를 사용자 정의 매개변수를 포함하는 도구로 변환
weather_tool = weather_agent.as_tool(
    name="WeatherLookup",
    description="모든 위치의 날씨 정보를 검색합니다 🔍☀️",
    arg_name="query",
    arg_description="날씨 질의 또는 위치 🌍"
)

main_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨",
    tools=weather_tool
)

result = await main_agent.run("제주의 날씨는 어떤가요? 🌴☀️")
print(result.text)